In [ ]:
import numpy as np
import pandas as pd
import polarDensity_helper as pc
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterExponent


In [ ]:
def polar_plot(data_in, theta, radius, chains_groups, helices_lwr, helices_upr, vmax=2, vmid=1, vmin=0):
	# plots densities
	# data_in = array/list of density data
	# theta, radius = arrays of position bins
	# chains_groups = old name, really lipids to plot

    data_in = pc.sum_reps(data_in)
    fig = plt.figure(figsize=(10,10))
    gs1=gridspec.GridSpec(len(chains_groups),2,wspace=.15, hspace=0.15)
    plt.rcParams.update({'font.size': 10})
    norm1 = pc.MidpointNormalize(midpoint=vmid,vmin=vmin,vmax=vmax)
    cmap = plt.cm.RdBu_r#PuOr
    cmap.set_bad(color='black')
    grid = 0
    #chains_up, chains_lo = prot_coord()
    sub = ["g",'m','grey','green','cyan']

    for cg in chains_groups:
        for leaf in data_in.columns:
            ax = plt.subplot(gs1[grid],projection="polar")
            toplot = data_in.at[cg,leaf]
            s = ax.pcolormesh(theta, 
                              radius, 
                              toplot,
                              cmap=cmap,
                              norm=norm1,
                              zorder=0,
                              )
            s.set_edgecolor('face')
            if grid%2==0:
                ax.set_ylabel(cg)
            if grid < 2:
                ax.set_title(leaf)
                
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            grid = grid + 1
            
            if leaf=="Outer":
                helices = helices_upr
            else:
                helices = helices_lwr

            for i,pro in enumerate(helices[:5]):
               ax.scatter(np.deg2rad(pro[1::2]),
                          pro[::2],
                          edgecolor=sub[i],
                          facecolors=sub[i],
                          linewidth=1,
                          zorder=2, 
                          s=np.shape(data_in)[0]*10,
                          )


    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.21, .89, 0.5, 0.008])
    fig.colorbar(s, cax=cbar_ax,ticks=np.linspace(vmin,vmax,5),orientation="horizontal")

    plt.tight_layout()
    return fig, ax

In [ ]:
'''
memb (str) should be synps or oocyt
ddg (bool) determines if ddg analysis called
lipids (str) points to a file to check lipids
enrich (bool) tells script to check enrichment
'''

# initialization

global Ntheta
Ntheta = 30

file_title1 = "polar1_"
density = None
chol_bind = None
w3_bind = None
rat = None
chains_groups = ["POPG", "POPE"]
lipids = chains_groups
col = []
#chains_up, chains_lo = pc.prot_coord()
enrich = True

# get files to use
root = Path("..")
files = root.glob("*.dat*.dat")

leaflets = ['low', 'upp']

file_list = list(files)

In [ ]:
enrichments = pd.DataFrame(index=chains_groups, columns=leaflets)
counts = pd.DataFrame(index=chains_groups, columns=leaflets)

idx = 0
for fl in file_list:
    if idx == 0:
        rad, dr, dth, theta, radius, frames = pc.Coord_Get(fl, Ntheta)

    filename = fl.name

    tmp_chain = filename.split('.')[0]
    tmp_nm = filename.split('.')[2]

    # This is a hack. The above part does not have a "flexible"
    # method to consider sim type (a, b ...)
    idx+=1
    toadd = np.loadtxt(fl, skiprows=1)
    toadd = toadd[:,3:-1]
    counts.at[tmp_chain,tmp_nm] = toadd
    enrichments.at[tmp_chain,tmp_nm] = pc._analysis_call_(fl, radius, dr, dth, frames, enrich=enrich)


In [ ]:
fig, ax = polar_plot(enrichments, theta, radius, chains_groups)
plt.savefig("old_enrichment.pdf")
plt.show()

In [ ]:
A = radius*dr*dth
densities = counts.copy()
for lip in lipids:
    for leaf in leaflets:
        densities.at[lip, leaf] = counts.loc[lip, leaf]/A

fig, ax = polar_plot(densities, theta, radius, chains_groups, vmax=0.3, vmid=0.15, vmin=0)
plt.savefig("densities.pdf")
plt.show()

In [ ]:
enrichments = densities/0.049
polar_plot(enrichments, theta, radius, chains_groups)